# Medic Attends and Completes Medical School

## As a result they are issued with a Primary Medical Qualification

Note for the time being we are modelling the medic as an ACA-Py agent although a more realistic scenario would use a mobile wallet.

![Zoom Cred](../zoom-cred-deps.png)

# This would likely be a Mobile Application Medics would need to Download

We represent the medic within notebooks for simplicity within this POC

![](https://tykn.tech/wp-content/uploads/2020/11/Tykn-Ana-App.png)

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://healthcare-professional-agent:3021 and an api key of MyMedicalAPIKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [4]:
listeners = []

In [5]:
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        print("Credential Stored\n")
        credential = payload["credential"]
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
    


    
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)

In [6]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
        

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [7]:
agent_controller.register_listeners(listeners)

## Establish a Connection

Must establish connection with issuer before being able to receive credential. Holder modeled as invitee in this case. See recipes/connection.

In [8]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '7f5c7343-8a24-4aca-b147-52942c3c0456', 'serviceEndpoint': 'http://ebf7-82-33-73-116.ngrok.io', 'label': 'Edinburgh Medical School', 'recipientKeys': ['7Vq6r3woRoGe8SsQtPggrB4SMdeSuxbNMPY3yspHFKg4']}

In [9]:
auto_accept=False
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a466d8a-ea65-4f64-985a-234de86fcdb2
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a466d8a-ea65-4f64-985a-234de86fcdb2
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a466d8a-ea65-4f64-985a-234de86fcdb2
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  3a466d8a-ea65-4f64-985a-234de86fcdb2
State :  active
Rout

In [ ]:
## TODO: Verify is Trusted Medical School?

## Medical Student Request Credential From Offer

The medical student can decide whether the attributes being offered are accurate, before requesting the final signature on a PMQ credential containing theseattributes.


In [10]:
state="offer_received"
records_response = await agent_controller.issuer.get_records(connection_id=connection_id, state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]


## Medic Verifies Details in Credential Offer and Requests When Satsified

In [11]:
response = await agent_controller.issuer.send_request_for_record(record_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 3a466d8a-ea65-4f64-985a-234de86fcdb2
Credential exchange ID : 46af9b90-9951-4ea2-98dc-d75b3c3e09de
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 3a466d8a-ea65-4f64-985a-234de86fcdb2
Credential exchange ID : 46af9b90-9951-4ea2-98dc-d75b3c3e09de
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


## Medic Stores Received PMQ for later use


In [12]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "PMQ"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 3a466d8a-ea65-4f64-985a-234de86fcdb2
Credential exchange ID : 46af9b90-9951-4ea2-98dc-d75b3c3e09de
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored


The referent acts as the identifier for retrieving the raw credential from the wallet
Referent PMQ


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [13]:
await agent_controller.terminate()